#1. Instalación / importación de librerías

In [1]:
# Ejecuta esta celda PRIMERO
!pip install -q -U langchain==0.3.0 langchain-community==0.3.0 langchain-google-genai==2.0.0 langchainhub==0.1.20 pysentimiento joblib google-generativeai

In [2]:
#!pip install --upgrade -q google-generativeai langchain-google-genai

In [3]:
import langchain
print(f"Versión cargada: {langchain.__version__}")

Versión cargada: 0.3.0


In [4]:
# 2. Descargar modelo de Spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 21.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import nltk # PLN
import spacy # Lemantización
from langchain_google_genai import ChatGoogleGenerativeAI # Nos permite utilizar un modelo de IA generativa de Google
from langchain.tools import tool #Importa el decorador para manipular funciones para el agente LangChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder # Nos permite definir instrucciones a considerar en el chat
from langchain_core.messages import HumanMessage, SystemMessage # Nos permite definir el contenido para el rol del sistema y el mensaje de usuario
from langchain.agents import create_react_agent, AgentExecutor # Importar la clase AgentExecutor
from langchain import hub # Nos permite descargar modelos pre-entrenados
import joblib # Librería para guardar modelos entrenados
from google.colab import files # Librería para descargar archivos generados
import sys # Manipulación de pahts
from google.colab import userdata # Llamada de secretos


#2. Descarga del repositorio github

In [6]:
# Desargamos el respositorio donde tenemos el código (Lo vamos a emplear para ejecutar la clase Preprocessor.py)
!git clone https://github.com/rsolis-utamed/pln_practica.git

fatal: destination path 'pln_practica' already exists and is not an empty directory.


In [7]:
#Actualizamos el el contenido del repositorio (si procede)
!cd /content/pln_practica && git pull

Already up to date.


#3. Definición de paths y descarga de modelos generados y script de procesamiento en el notebook "modelado"

In [8]:
# Definimos un path para poder trabajar con scripts python subidos al respositorio
sys.path.append('/content/pln_practica/code')
sys.path.append('/content/pln_practica/models')

In [9]:

clf_textb_sent = joblib.load('/content/pln_practica/models/text_blob_model.joblib')
cld_lda_topics = joblib.load('/content/pln_practica/models/lda_topics_model.joblib')
vectorizer = joblib.load('/content/pln_practica/models/vectorizador_tfidf.joblib')

In [10]:
# Descargar recursos de NLTK necesarios
nltk.download('punkt') # Recurso encargado de tokenizar (división del texto en unidades más pequeñas, normalmente palabras)
nltk.download('stopwords') # Recurso que contiene palabras muy comunes que no aportan significado temático por sí solas.
nltk.download('punkt_tab') # Recurso que asegura la compatibilidad del sistema de división de palabras y frases.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
# Ejecutar en terminal antes: python -m spacy download es_core_news_sm
try:
    nlp = spacy.load("es_core_news_sm") # Descargamos un modelo pre-entrenado en español
except:
    print("Spacy model non found. Execute: python -m spacy download es_core_news_sm")

In [12]:
# Palabras encontradas en topicos generados anteriormente (en pruebas previas) que se ha considearo que no aportan valor. Se incorporan dentro de
# el resto de stopwords
ruido_nuevo = ['él', 'decir', 'dejar', 'cada', 'dar', 'año', 'pasar','alguno', 'claro', 'igual', 'siempre', 'vez', 'medio']

In [13]:
# Importamos la clase propia preprocessor del reposistorio github
from Preprocessor import Preprocessor
preprocesador=Preprocessor(nlp,ruido_nuevo)


version 2.1.0


# 4. Definición de funciones que encapsulan la llamada de los modelos

In [14]:

@tool
def det_sentimiento(text: str) -> str:
    """
    Analiza el sentimiento de un texto dado (positivo, negativo, neutro).
    Utiliza el modelo de análisis de sentimiento pre-entrenado cargado como 'clf_textb_sent'.
    """
    try:
        preprocessed_text = preprocesador.toPreprocessText(text)
        vec_text = vectorizer.transform([preprocessed_text])[0]

        if not preprocessed_text:
            return "No se pudo procesar el texto para análisis de sentimiento. El texto preprocesado resultó vacío o nulo."
        sentiment_prediction = clf_textb_sent.predict([vec_text])
        return f"El sentimiento del texto es: {sentiment_prediction}"
    except Exception as e:
        return f"Error al analizar el sentimiento: {e}"

@tool
def det_topic(text: str) -> str:
    """
    Identifica el tópico principal de un texto dado.
    Utiliza el modelo LDA de tópicos cargado como 'cld_lda_topics' y el vectorizador TF-IDF 'vectorizer'.
    """
    try:
        preprocessed_text = preprocesador.toPreprocessText(text)

        if not preprocessed_text:
            return "No se pudo procesar el texto para análisis de tópicos. El texto preprocesado resultó vacío o nulo."
        text_vectorized = vectorizer.transform([preprocessed_text])
        topic_distribution = cld_lda_topics.transform(text_vectorized)[0]
        dominant_topic_idx = np.argmax(topic_distribution)

        # Nota: Los nombres de los tópicos no están definidos en el contexto actual,
        # así que se devuelve el índice del tópico dominante y su probabilidad.
        return f"El tópico principal del texto es el Tópico {dominant_topic_idx} con una probabilidad de {topic_distribution[dominant_topic_idx]:.2f}"
    except Exception as e:
        return f"Error al analizar los tópicos: {e}"

In [15]:
tools=[det_sentimiento,det_topic]

# 5. Definición de agente LangChain y llamada de modelo llm de google gemini

In [16]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [17]:
# Definimos el LLM (Google Gemini)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                             temperature=0.7,
                             top_p=0.85,
                             google_api_key=GOOGLE_API_KEY,
                             version="v1"
)

In [18]:
# 2. Definir el Rol y las Instrucciones (Prompt)
instrucciones_sistema ="""Eres un Asistente Especialista en Análisis de Opiniones de la ciudad de Málaga.
Tu única misión es clasificar el SENTIMIENTO y el TÓPICO, empleando las funciones propias {tools}, de los mensajes que te den.

REGLAS CRÍTICAS:
1. Solo puedes responder preguntas relacionadas con el análisis de los textos proporcionados.
2. Si el usuario te pregunta sobre recetas, política general, o temas que no sean analizar mensajes,
   responde educadamente: 'Lo siento, como analista de opiniones de Málaga, solo puedo ayudarte con el sentimiento y tópicos de mensajes'.
3. Siempre usa las herramientas proporcionadas para dar una respuesta técnica basada en los modelos .joblib."""

In [19]:

#D Definimos la plantilla para el prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", instrucciones_sistema),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),  # Variable principal
    MessagesPlaceholder(variable_name="agent_scratchpad"), # Espacio para que el agente piense
])

In [20]:
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.12/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [21]:
# Creamos un agente al que le pasamos el modelo llm, los modelos propios para polaridad y tópicos y la plantilla de prompt
agent=create_react_agent(llm,tools,prompt)

In [22]:
# Creamos el Ejecutor (El cuerpo que mueve al agente)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True, # Ponemos True para ver cómo piensa
    handle_parsing_errors=True
)

In [27]:
input_chat="El servicio de autobuses es terrible y el conductor fue grosero"


In [29]:
# Ejecución del agente que devuelve la polaridad y clasificación del tópico en función del texto pasado como prompt
agent_executor.invoke({"input": input_chat})



> Entering new AgentExecutor chain...
Action: det_sentimiento
Action Input: El servicio de autobuses es terrible y el conductor fue grosero   Parsing to lower...
   Applying regular expresions...
   Tokenizing text...
   Lemmantizing text...
   Deleting stop words in text...
Error al analizar el sentimiento: Expected 2D array, got 1D array instead:
array=[<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1 stored elements and shape (1, 210)>            ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.Thought: The `det_sentimiento` tool failed with an internal error related to array reshaping. This indicates that the tool is not correctly preparing the input for its underlying sentiment analysis model. As `det_sentimiento` is the only tool available for sentiment analysis, and it is not functioning correctly for the given input, I cannot determine the sentiment to answer the que

{'input': 'El servicio de autobuses es terrible y el conductor fue grosero',
 'output': 'No se puede determinar el sentimiento debido a un error interno en la herramienta `det_sentimiento`. La herramienta falló con el mensaje: "Expected 2D array, got 1D array instead: array=[<Compressed Sparse Row sparse matrix of dtype \'float64\' with 1 stored elements and shape (1, 210)> ]. Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample."'}